# Back up some issue to be further explored

### IV calculated using mibian is different from what I have drafted by myself

In [1]:
import pandas as pd
import numpy as np
import mibian
import math
from scipy.stats import norm
import matplotlib.pyplot as plt

In [2]:
OptionList = pd.read_csv("nifty_311221_070122_1min.csv",index_col=0)
OptionList.head()

,date,opt_price,volume_x,opt_symbol,fut_price,volume_y,fut_symbol,name,expiry,strike,instrument_type,ts,days_to_expiry
index,,,,,,,,,,,,,
0,2021-12-31 09:15:00+05:30,493.10,5400,NIFTY22JAN17000CE,17311.0,174100,NIFTY22JANFUT,NIFTY,2022/1/27 15:30,17000,CE,2021/12/31 9:15,27.260764
1,2021-12-31 09:15:00+05:30,82.95,0,NIFTY22JAN17850CE,17311.0,174100,NIFTY22JANFUT,NIFTY,2022/1/27 15:30,17850,CE,2021/12/31 9:15,27.260764
2,2021-12-31 09:15:00+05:30,600.00,0,NIFTY22JAN17850PE,17311.0,174100,NIFTY22JANFUT,NIFTY,2022/1/27 15:30,17850,PE,2021/12/31 9:15,27.260764
3,2021-12-31 09:15:00+05:30,74.95,3150,NIFTY22JAN17900CE,17311.0,174100,NIFTY22JANFUT,NIFTY,2022/1/27 15:30,17900,CE,2021/12/31 9:15,27.260764
4,2021-12-31 09:15:00+05:30,631.20,0,NIFTY22JAN17900PE,17311.0,174100,NIFTY22JANFUT,NIFTY,2022/1/27 15:30,17900,PE,2021/12/31 9:15,27.260764


In [5]:
callsATM = OptionList[(OptionList['instrument_type']=='CE')&(abs(OptionList['strike'] - OptionList['fut_price']) <= 25)]
#callsATM.head()
print(callsATM.count())

date               2271
opt_price          2271
volume_x           2271
opt_symbol         2271
fut_price          2271
volume_y           2271
fut_symbol         2271
name               2271
expiry             2271
strike             2271
instrument_type    2271
ts                 2271
days_to_expiry     2271
dtype: int64


In [6]:
putsATM = OptionList[(OptionList['instrument_type']=='PE')&(abs(OptionList['strike'] - OptionList['fut_price']) <= 25)]
#putsATM.describe()
print(putsATM.count())

date               2271
opt_price          2271
volume_x           2271
opt_symbol         2271
fut_price          2271
volume_y           2271
fut_symbol         2271
name               2271
expiry             2271
strike             2271
instrument_type    2271
ts                 2271
days_to_expiry     2271
dtype: int64


In [7]:
tmp = mibian.BS([42,40,0.1,132.5],callPrice = 4.76)
#tmp = mibian.BS([1.4565, 1.45, 1, 30], callPrice=0.0359)
tmp.impliedVolatility

37.2314453125

In [8]:
tmp = mibian.BS([42,40,0.1,132.5],volatility = 37.23144)
#tmp = mibian.BS([1.4565, 1.45, 1, 30],volatility = 19.22607421875)
tmp.callPrice

4.7552259935315035

In [9]:
def CallOptionPrice(S0,K,r,T,sigma):
    d1 = (math.log(S0/K)+(r+sigma*sigma/2)*T)/(sigma*math.sqrt(T))
    #print("d1: ",d1)
    d2 = d1 - sigma*math.sqrt(T)
    #print("d2: ",d2)
    c = S0*norm.cdf(d1) - K*math.exp(-r*T)*norm.cdf(d2)
    #print(c)
    return c

def CallIV(S0,K,ir,T,callPrice):
    l = 0.0
    r = 1.0
    while(r-l>0.00005):
        mid = l+(r-l)/2
        tmp = CallOptionPrice(S0,K,ir,T,mid)
        print("tmp: ",tmp,"l: ",l,"r: ",r,"mid: ",mid)
        if(tmp<callPrice):
            l = mid
        else:
            r = mid
    return mid

In [10]:
## compare mibian and self made IV calcualtor
tmp = mibian.BS([172.17,172.25,0.3,5],callPrice = 2.37)
print(tmp.impliedVolatility)
print(CallIV(172.17,85,0.3,433,90.73))

29.9072265625
tmp:  172.17 l:  0.0 r:  1.0 mid:  0.5
tmp:  172.17 l:  0.0 r:  0.5 mid:  0.25
tmp:  172.17 l:  0.0 r:  0.25 mid:  0.125
tmp:  172.17 l:  0.0 r:  0.125 mid:  0.0625
tmp:  172.17 l:  0.0 r:  0.0625 mid:  0.03125
tmp:  172.17 l:  0.0 r:  0.03125 mid:  0.015625
tmp:  172.17 l:  0.0 r:  0.015625 mid:  0.0078125
tmp:  172.17 l:  0.0 r:  0.0078125 mid:  0.00390625
tmp:  172.17 l:  0.0 r:  0.00390625 mid:  0.001953125
tmp:  172.17 l:  0.0 r:  0.001953125 mid:  0.0009765625
tmp:  172.17 l:  0.0 r:  0.0009765625 mid:  0.00048828125
tmp:  172.17 l:  0.0 r:  0.00048828125 mid:  0.000244140625
tmp:  172.17 l:  0.0 r:  0.000244140625 mid:  0.0001220703125
tmp:  172.17 l:  0.0 r:  0.0001220703125 mid:  6.103515625e-05
tmp:  172.17 l:  0.0 r:  6.103515625e-05 mid:  3.0517578125e-05
3.0517578125e-05


In [11]:
print(CallIV(172.17,172.25,0.3,5/252,2.37))

tmp:  5.3090349806033 l:  0.0 r:  1.0 mid:  0.5
tmp:  2.912366612550258 l:  0.0 r:  0.5 mid:  0.25
tmp:  1.7352883788296936 l:  0.0 r:  0.25 mid:  0.125
tmp:  2.319087801023656 l:  0.125 r:  0.25 mid:  0.1875
tmp:  2.615046246510701 l:  0.1875 r:  0.25 mid:  0.21875
tmp:  2.4668575751880297 l:  0.1875 r:  0.21875 mid:  0.203125
tmp:  2.3929140771037254 l:  0.1875 r:  0.203125 mid:  0.1953125
tmp:  2.355985395684229 l:  0.1875 r:  0.1953125 mid:  0.19140625
tmp:  2.3744459673962126 l:  0.19140625 r:  0.1953125 mid:  0.193359375
tmp:  2.3652147250486735 l:  0.19140625 r:  0.193359375 mid:  0.1923828125
tmp:  2.3698301089019367 l:  0.1923828125 r:  0.193359375 mid:  0.19287109375
tmp:  2.372137979042307 l:  0.19287109375 r:  0.193359375 mid:  0.193115234375
tmp:  2.370984029167616 l:  0.19287109375 r:  0.193115234375 mid:  0.1929931640625
tmp:  2.3704070653301272 l:  0.19287109375 r:  0.1929931640625 mid:  0.19293212890625
tmp:  2.370118586189463 l:  0.19287109375 r:  0.19293212890625 mid

In [12]:
CallOptionPrice(172.17,85,0.3,433,0.25)

172.17

### is there a better way to calculate IV of a large data set?